<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

## Introduction to assignment - Scraping data

The goal of this assignment is to show neighborhoods in Toronto through exploring, segmenting and clustering. First, the data will be scraped from Wikipedia page. After that, it will be preprocesed and cleaned, prepered for futher work.

## Libraries

Downloading all dependencies that will be required.

In [1]:
import pandas as pd #library for data analsysis
import numpy as np #library to handle data in a vectorized manner
import requests #library to handle requests
import urllib.request #library to use to open URLs
import bs4
from bs4 import BeautifulSoup #library to import the BeautifulSoup library so we can parse HTML and XML documents
print('Libraries imported.')

Libraries imported.


## Downloading dataset and reading/cleaning/preprocesing data

Using BeautifulSooup, data'll be grabed from the website and prepared to fit into dataframe.

In [2]:
page = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
# transform page with BeautifulSoup
soup = BeautifulSoup(page.content, 'html.parser')

# find table with specific class definition - after page source manual examination
gdp = soup.find_all("table", attrs={"class": "wikitable sortable"})
print("Number of tables on site: ", len(gdp))
table = gdp[0]  # first and only table that is selected is one that interest us

postcodes_dict = {}  # initialize an empty dictionary to save the data in
skip_first = False  # first row in table needs to be skipped, since it is header

for table_cell in table.find_all('tr'):
    try:
        if skip_first:
            row_data = table_cell.find_all('td')
            # let us first investigate shall we skip the record
            borough = row_data[1].text.strip('\n')
            if borough != 'Not assigned':  # skip all that are not assigned
                neighborhoods = row_data[2].text.strip('\n')
                if neighborhoods == 'Not assigned':  # if borough exists but no neighborhood, they are the same
                    neighborhoods = borough
                postal_code = row_data[0].text.strip('\n')
                if postal_code in postcodes_dict.keys():  # join neighborhoods_data if postal code already exists
                    postcodes_dict[postal_code]['neighborhoods'] = \
                        postcodes_dict[postal_code]['neighborhoods'] + ', ' + neighborhoods
                else:  # everything is fine, add new postal_code key into dictionary
                    postcodes_dict[postal_code] = {}
                    # add borough and neighborhood to dictionary
                    postcodes_dict[postal_code]['borough'] = borough
                    postcodes_dict[postal_code]['neighborhoods'] = neighborhoods
        else:
            skip_first = True
    except SystemError as error:
        print(error)

    

Number of tables on site:  1


## Put data into dataframe

Preprocesed data'll load into dataframe and printed as a table.

In [3]:
# empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
data_toronto = pd.DataFrame(columns=columns)
# populate dataframe
for ind, postal_code in enumerate(postcodes_dict):
    borough = postcodes_dict[postal_code]['borough']
    neighborhood = postcodes_dict[postal_code]['neighborhoods']
    data_toronto = data_toronto.append({"PostalCode": postal_code,
                                        "Borough": borough,
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

In [4]:
# print number of rows of dataframe
print(data_toronto.shape[0])
print()
data_toronto

103



,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [5]:
print("Shape is:" , data_toronto.shape)

Shape is: (103, 3)
